In [1]:
## One shot learning for audio with siamese network. 
# author: Saltanat Khalyk
# modified code from @ttchengab ~~~ github.com

In [2]:
import torch
import torch
import torchaudio
import torchaudio.functional as F
import torchaudio.transforms as T
import matplotlib.pyplot as plt

In [3]:
torch.cuda.set_device(0)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
from os import walk
import torch

import matplotlib.pyplot as plt
import os

import torchaudio
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset,Dataset, DataLoader, random_split
import pandas as pd
import numpy as np
import random
#from torch.utils.data import TensorDataset, DataLoader, Dataset
import torch.utils.data as data_utils
import os

In [6]:
# creating the pairs of images for inputs, same character label = 1, vice versa
class sound_Dataset(Dataset):
    def __init__(self, categories, root_dir, transform=None):
        self.categories = categories
        self.root_dir = root_dir
        self.transform = transform
        
        
        audio_names =[]
        for subdir, dirs, files in os.walk(self.root_dir):
            for file in files:
                filepath =subdir+os.sep+file
                audio_names.append(filepath)
        self.audio_names = audio_names
        
    def __len__(self):
        return len(self.audio_names)
    def __getitem__(self, idx):
        audio1 = None
        audio2 = None
        label = None
        
        audio_names =[]
        for subdir, dirs, files in os.walk(self.root_dir):
            for file in files:
                filepath =subdir+os.sep+file
                audio_names.append(filepath)
        self.audio_names = audio_names
        
        class_name = self.audio_names[idx].split('\\')[1]
        list_categories = ['asthma', 'allergies', 'pneumonia', 'covid','other']
        channel_dict = {'asthma': 0, 'allergies': 1, 'pneumonia': 2, 'covid':3,'other':4}
        label = channel_dict.get(class_name)
        
        #audio1, sample_rate = torchaudio.load('D:/COVID_COUGH_SOUNDS/1second_chunks_458/balanced_1s_not_aug/one_shot1sec/train/covid/1629490127741_3.wav')
        n_fft = 1024
        win_length = None
        hop_length = 512
        n_mels = 128

        
        
        
        
        
        
        
        if idx % 2 == 0: # select the same character for both images
            category = random.choice(self.categories)
            character = random.choice(category[1])
            #category = random.choice(list_categories)
            audioDir = str(self.root_dir) + str(category[0]) 
            audio1Name = random.choice(os.listdir(audioDir))
            audio2Name = random.choice(os.listdir(audioDir))
            audio1, sample_rate = torchaudio.load(str(audioDir) + os.sep + str(audio1Name)) 
            audio2, sample_rate = torchaudio.load(str(audioDir) + os.sep + str(audio2Name))
            mel_spectrogram = T.MelSpectrogram(
            sample_rate=sample_rate,
            n_fft=n_fft,
            win_length=win_length,
            hop_length=hop_length,
            center=True,
            pad_mode="reflect",
            power=2.0,
            norm="slaney",
            onesided=True,
            n_mels=n_mels,
            mel_scale="htk",)
            
            
            

            melspec1 = mel_spectrogram(audio1)
            melspec2 = mel_spectrogram(audio2)
            
            #print(audioDir + os.sep + audio1Name)
            #print(audioDir + os.sep + audio2Name)
            label = 1.0
        else: # select a different character for both images
            category1, category2 = random.choice(self.categories), random.choice(self.categories)
            category1, category2 = random.choice(self.categories), random.choice(self.categories)
            character1, character2 = random.choice(category1[1]), random.choice(category2[1])
            audioDir1, audioDir2 = str(self.root_dir) + str(category1[0]), str(self.root_dir) + str(category2[0])
            audio1Name = random.choice(os.listdir(audioDir1))
            audio2Name = random.choice(os.listdir(audioDir2))
            while audio1Name == audio2Name:
                audio2Name = random.choice(os.listdir(audioDir2))
            label = 0.0
            audio1, sample_rate = torchaudio.load(str(audioDir1) + os.sep + str(audio1Name)) 
            audio2, sample_rate = torchaudio.load(str(audioDir2) + os.sep+ str(audio2Name))
            
            mel_spectrogram = T.MelSpectrogram(
            sample_rate=sample_rate,
            n_fft=n_fft,
            win_length=win_length,
            hop_length=hop_length,
            center=True,
            pad_mode="reflect",
            power=2.0,
            norm="slaney",
            onesided=True,
            n_mels=n_mels,
            mel_scale="htk",)
            
            melspec1 = mel_spectrogram(audio1)
            melspec2 = mel_spectrogram(audio2)
#         plt.imshow(img1)
        if self.transform:
            melspec1 = self.transform(melspec1)
            melspec2 = self.transform(melspec2)
        return melspec1, melspec2, torch.from_numpy(np.array([label], dtype=np.float32)) 

In [7]:
path_train = 'D:/COVID_COUGH_SOUNDS/1second_chunks_458/balanced_1s_not_aug/one_shot1sec/train/'
path_test = 'D:/COVID_COUGH_SOUNDS/1second_chunks_458/balanced_1s_not_aug/one_shot1sec/test/'
path_val = 'D:/COVID_COUGH_SOUNDS/1second_chunks_458/balanced_1s_not_aug/one_shot1sec/val/'
#root_dir = path_train
categories_train = [[folder, os.listdir(path_train +'/'+ folder)] for folder in os.listdir(path_train)  if not folder.startswith('.') ]
#path_val = 'D:/COVID_COUGH_SOUNDS/1second_chunks_458/balanced_1s_not_aug/one_shot1sec/val/'
categories_val = [[folder, os.listdir(path_val +'/'+ folder)] for folder in os.listdir(path_val)  if not folder.startswith('.') ]


train_set = sound_Dataset(categories_train, path_train, transform =None)
val_set = sound_Dataset(categories_val, path_val, transform =None)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, num_workers=0)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=1, num_workers=0)

shape of melspec1 for audio 1 - torch.Size([32, 1, 128, 44])


shape of melspec2 for audio 2 - torch.Size([32, 1, 128, 44])

labels of if audio1=audio2 then 1, else 0 : torch.Size([32, 1])

In [8]:
# creates n-way one shot learning evaluation
class NWayOneShotEvalSet(Dataset):
    def __init__(self, categories, root_dir,  numWay, transform=None):
        self.categories = categories
        self.root_dir = root_dir
        #self.setSize = setSize
        self.numWay = numWay
        self.transform = transform
        audio_names =[]
        for subdir, dirs, files in os.walk(self.root_dir):
            for file in files:
                filepath =subdir+os.sep+file
                audio_names.append(filepath)
        self.audio_names = audio_names
        
    def __len__(self):
        return len(self.audio_names)
    def __getitem__(self, idx):
        
        # find one main image
        n_fft = 1024
        win_length = None
        hop_length = 512
        n_mels = 128
        
        category = random.choice(self.categories)
        character = random.choice(category[1])
        audioDir = str(self.root_dir) + str(category[0])
        audioName = random.choice(os.listdir(audioDir))
        mainAudio, sample_rate = torchaudio.load(str(audioDir) + os.sep + str(audioName))
        
        mel_spectrogram = T.MelSpectrogram(
            sample_rate=sample_rate,
            n_fft=n_fft,
            win_length=win_length,
            hop_length=hop_length,
            center=True,
            pad_mode="reflect",
            power=2.0,
            norm="slaney",
            onesided=True,
            n_mels=n_mels,
            mel_scale="htk",)
            
            
            

        melspec1 = mel_spectrogram(mainAudio)
        
        
        # print(imgDir + '/' + imgName)
        if self.transform:
            melspec1 = self.transform(melspec1)
        
        # find n numbers of distinct images, 1 in the same set as the main
        testSet = []
        label = np.random.randint(self.numWay)
        for i in range(self.numWay):
            testAudioDir = audioDir
            testAudioName = ''
            if i == label:
                testAudioName = random.choice(os.listdir(audioDir))
            else:
                testCategory = random.choice(self.categories)
                testCharacter = random.choice(testCategory[1])
                testAudioDir = self.root_dir + testCategory[0]
                while testAudioDir == audioDir:
                    testAudioDir = self.root_dir + testCategory[0]
                testAudioName = random.choice(os.listdir(testAudioDir))
            testAudio, sample_rate = torchaudio.load(str(testAudioDir) + os.sep + str(testAudioName))
            
            mel_spectrogram = T.MelSpectrogram(
            sample_rate=sample_rate,
            n_fft=n_fft,
            win_length=win_length,
            hop_length=hop_length,
            center=True,
            pad_mode="reflect",
            power=2.0,
            norm="slaney",
            onesided=True,
            n_mels=n_mels,
            mel_scale="htk",)
            
            melspec2 = mel_spectrogram(testAudio)
            
            
            if self.transform:
                melspec2 = self.transform(melspec2)
            
            testSet.append(melspec2)
        
        return melspec1, testSet, torch.from_numpy(np.array([label], dtype = int))

In [9]:
numWay =4
label = np.random.randint(numWay)
print(label)

1


In [17]:
root_dir = path_train
numWay=2
categories = [[folder, os.listdir(path_train +'/'+ folder)] for folder in os.listdir(path_train)  if not folder.startswith('.') ]
category = random.choice(categories)
#print('category',category, '/n')
character = random.choice(category[1])
print('character:' , character)
audioDir = str(root_dir) + str(category[0])
print('audio DIR: ',audioDir)
audioName = random.choice(os.listdir(audioDir))
print('Audio name',audioName)
mainAudio, sample_rate = torchaudio.load(str(audioDir) + os.sep + str(audioName))
print('main audio', mainAudio.size())
n_fft = 1024
win_length = None
hop_length = 512
n_mels = 128        
mel_spectrogram = T.MelSpectrogram(
    sample_rate=sample_rate,
    n_fft=n_fft,
    win_length=win_length,
    hop_length=hop_length,
    center=True,
    pad_mode="reflect",
    power=2.0,
    norm="slaney",
    onesided=True,
    n_mels=n_mels,
    mel_scale="htk",)
            
            
            

melspec1 = mel_spectrogram(mainAudio)
print('Melspc:', melspec1.size())       
        
        # print(imgDir + '/' + imgName)
#if self.transform:
    #melspec1 = self.transform(melspec1)
        
        # find n numbers of distinct images, 1 in the same set as the main
testSet = []
label = np.random.randint(numWay)
print(label)
for i in range(numWay):
    testAudioDir = audioDir
    testAudioName = ''
    if i == label:
        print('label==i', label, i)
        testAudioName = random.choice(os.listdir(audioDir))
        print('label equal:',testAudioName, testAudioDir)
    else:
        testCategory = random.choice(categories)
        testCharacter = random.choice(testCategory[1])
        testAudioDir = root_dir + testCategory[0]
        print('testAudioDir: ',testAudioDir)
        print('label not equal i', label, i)
        while testAudioDir == audioDir:
            testAudioDir = root_dir + testCategory[0]
        testAudioName = random.choice(os.listdir(testAudioDir))
        print('test AUDIO name label equal', testAudioName, testAudioDir)
    testAudio, sample_rate = torchaudio.load(str(testAudioDir) +os.sep+ str(testAudioName))
            
    mel_spectrogram = T.MelSpectrogram(
        sample_rate=sample_rate,
        n_fft=n_fft,
        win_length=win_length,
        hop_length=hop_length,
        center=True,
        pad_mode="reflect",
        power=2.0,
        norm="slaney",
        onesided=True,
        n_mels=n_mels,
        mel_scale="htk",)
            
    melspec2 = mel_spectrogram(testAudio)
            
            
            
    testSet.append(melspec2)
    print(testSet)

character: 1605083054207_5.wav
audio DIR:  D:/COVID_COUGH_SOUNDS/1second_chunks_458/balanced_1s_not_aug/one_shot1sec/train/other
Audio name 1605574542532_4.wav
main audio torch.Size([1, 22050])
Melspc: torch.Size([1, 128, 44])
1
testAudioDir:  D:/COVID_COUGH_SOUNDS/1second_chunks_458/balanced_1s_not_aug/one_shot1sec/train/allergies
label not equal i 1 0
test AUDIO name label equal 1604912115920_2.wav D:/COVID_COUGH_SOUNDS/1second_chunks_458/balanced_1s_not_aug/one_shot1sec/train/allergies
[tensor([[[5.5670e-05, 1.7147e-05, 1.5482e-05,  ..., 9.6861e-05,
          9.8773e-05, 9.5619e-05],
         [5.0993e-05, 4.3186e-05, 3.2470e-05,  ..., 1.7789e-04,
          9.8419e-05, 9.2526e-05],
         [4.5146e-05, 8.6817e-05, 6.1082e-05,  ..., 3.1514e-04,
          1.0118e-04, 9.0651e-05],
         ...,
         [1.6850e-09, 1.3878e-09, 1.4954e-09,  ..., 1.8272e-09,
          1.6344e-09, 2.5189e-08],
         [1.1052e-09, 1.9688e-09, 1.1421e-09,  ..., 1.4110e-09,
          1.8511e-09, 2.1684e-0

In [70]:
numWay = 1


path_train = 'D:/COVID_COUGH_SOUNDS/1second_chunks_458/balanced_1s_not_aug/one_shot1sec/train/'
path_test = 'D:/COVID_COUGH_SOUNDS/1second_chunks_458/balanced_1s_not_aug/one_shot1sec/test/'
#root_dir = path_train
categories = [[folder, os.listdir(path_test +'/'+ folder)] for folder in os.listdir(path_test)  if not folder.startswith('.') ]

test_set = NWayOneShotEvalSet(categories, path_test, numWay, transform=None)
test_loader = torch.utils.data.DataLoader(test_set, batch_size = 1, num_workers = 0, shuffle=True)

In [71]:
#for melspec1,testSet, labels in (test_loader):
        # get the inputs; data is a list of [inputs, labels]
        #for i, melspec2 in enumerate(testSet):
            #print(melspec2.size())

In [72]:
import torch.nn as nn
import torch.nn.functional as F

#### Questions:


(background infa: I have huge tensors for audio files, and they take a lot of memory,  so I want to reduce the size and extract only useful features to feed them to cnn)

how to extract audio features (in pytorch only)? 


1. extract mel frequency?
2. or use pretrained torchaudio.models (emformer, convTasNet) - is it useful when the audio is not speech to text conversion problem?
3. or use pretrained torchaudio.pipelines? (also not asr task, is it useful?)

what is the good approach (1.2.3)? or all of them are good/bad? or what other methods could u recommend?

#audioacousticfeaturextraction

In [73]:
import torch
import torch
import torchaudio
import torchaudio.functional as F
import torchaudio.transforms as T
import matplotlib.pyplot as plt
#model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
#model.eval()

In [74]:
## Extract mfcc features from audio files

## Can I use any other cnn for the siamese network? 
The original siamese net for one-shot-learning was for 4d input, whereas we have 3d. 
so I modified the network slightly, but the it took a lot of memory. For example can I try something smaller and apply sigmoid function in the end??

In [75]:
#Different network structures, the commented out are the different experimenting structures
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # Koch et al.
        # Conv2d(input_channels, output_channels, kernel_size)
        self.conv1 = nn.Conv2d(1, 2, 3) 
        self.conv2 = nn.Conv2d(2, 5, 3)  
        self.conv3 = nn.Conv2d(5, 10, 1)
        self.conv4 = nn.Conv2d(10, 16, 1)
        self.bn1 = nn.BatchNorm2d(2)
        self.bn2 = nn.BatchNorm2d(5)
        self.bn3 = nn.BatchNorm2d(10)
        self.bn4 = nn.BatchNorm2d(16)
        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(960,32)
        self.fcOut = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

        # VGG16
        # # dataiter = iter(train_loader)
        # # img1, img2, label = dataiter.next()
        # # print(img1.shape)
        # self.conv11 = nn.Conv2d(1, 64, 3) 
        # self.conv12 = nn.Conv2d(64, 64, 3)  
        # self.conv21 = nn.Conv2d(64, 128, 3)
        # self.conv22 = nn.Conv2d(128, 128, 3)
        # self.conv31 = nn.Conv2d(128, 256, 3) 
        # self.conv32 = nn.Conv2d(256, 256, 3)  
        # self.conv33 = nn.Conv2d(256, 256, 3)
        # self.pool = nn.MaxPool2d(2, 2)
        # self.fc1 = nn.Linear(256 * 8 * 8, 4096)
        # self.fc2 = nn.Linear(4096, 4096)
        # self.fcOut = nn.Linear(4096, 1)
        # self.sigmoid = nn.Sigmoid()
        # # x = self.conv11(img1)
        # # x = self.conv12(x)
        # # x = self.pool(x)
        # # x = self.conv21(x)
        # # x = self.conv22(x)
        # # x = self.pool(x)
        # # x = self.conv31(x)
        # # x = self.conv32(x)
        # # x = self.conv33(x)
        # # x = self.pool(x)
        # # print(x.shape)
    
    def convs(self, x):

        # Koch et al.
        # out_dim = in_dim - kernel_size + 1  
        #1, 105, 105
        x = torch.nn.functional.relu(self.bn1(self.conv1(x)))
        # 64, 96, 96
        x = torch.nn.functional.max_pool2d(x, (2,2))
        # 64, 48, 48
        x = torch.nn.functional.relu(self.bn2(self.conv2(x)))
        # 128, 42, 42
        x = torch.nn.functional.max_pool2d(x, (2,2))
        # 128, 21, 21
        x = torch.nn.functional.relu(self.bn3(self.conv3(x)))
        # 128, 18, 18
        x = torch.nn.functional.max_pool2d(x, (2,2))
        # 128, 9, 9
        x = torch.nn.functional.relu(self.bn4(self.conv4(x)))
        # 256, 6, 6
        return x

        # VGG16
        # x = F.relu(self.conv11(x))
        # x = F.relu(self.conv12(x))
        # x = F.max_pool2d(x, (2,2))
        # x = F.relu(self.conv21(x))
        # x = F.relu(self.conv22(x))
        # x = F.max_pool2d(x, (2,2))
        # x = F.relu(self.conv31(x))
        # x = F.relu(self.conv32(x))
        # x = F.relu(self.conv33(x))
        # x = F.max_pool2d(x, (2,2))
        # return x

    def forward(self, x1, x2):
        x1 = self.convs(x1)

        # Koch et al.
        x1 = x1.view(x1.size(0), -1)
        x1 = self.sigmoid(self.fc1(x1))

        # VGG16
        # x1 = x1.view(-1, 256 * 8 * 8)
        # x1 = self.fc1(x1)
        # x1 = self.sigmoid(self.fc2(x1))
        
        x2 = self.convs(x2)

        # Koch et al.
        x2 = x2.view(x2.size(0), -1)
        x2 = self.sigmoid(self.fc1(x2))

        # VGG16
        # x2 = x2.view(-1, 256 * 8 * 8)
        # x2 = self.fc1(x2)
        # x2 = self.sigmoid(self.fc2(x2))

        x = torch.abs(x1 - x2)
        x = self.fcOut(x)
        return x

In [76]:
class VGGSiameseNet(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv11 = nn.Conv2d(1, 64, 3) 
        self.conv12 = nn.Conv2d(64, 64, 3)  
        self.conv21 = nn.Conv2d(64, 128, 3)
        self.conv22 = nn.Conv2d(128, 128, 3)
        self.conv31 = nn.Conv2d(128, 256, 3) 
        self.conv32 = nn.Conv2d(256, 256, 3)  
        self.conv33 = nn.Conv2d(256, 256, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(256 * 8 * 8, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fcOut = nn.Linear(4096, 1)
        self.sigmoid = nn.Sigmoid()
    
    def convs(self, x):
        x = F.relu(self.conv11(x))
        x = F.relu(self.conv12(x))
        x = F.max_pool2d(x, (2,2))
        x = F.relu(self.conv21(x))
        x = F.relu(self.conv22(x))
        x = F.max_pool2d(x, (2,2))
        x = F.relu(self.conv31(x))
        x = F.relu(self.conv32(x))
        x = F.relu(self.conv33(x))
        x = F.max_pool2d(x, (2,2))
        return x

    def forward(self, x1, x2):
        x1 = self.convs(x1)
        x1 = x1.view(-1, 256 * 8 * 8)
        x1 = self.fc1(x1)
        x1 = self.sigmoid(self.fc2(x1))
        x2 = self.convs(x2)
        x2 = x2.view(-1, 256 * 8 * 8)
        x2 = self.fc1(x2)
        x2 = self.sigmoid(self.fc2(x2))
        x = torch.abs(x1 - x2)
        x = self.fcOut(x)
        return x

In [77]:
def evaluate(model, test_loader):
    with torch.no_grad():
        model.eval()
        correct = 0
        count = 0
        for melspec1, testSet, label in test_loader:
            melspec1 = melspec1.to(device)
            predVal = 0
            pred = -1
            
            # determine which category an image belongs to
            for i, melspec2 in enumerate(testSet):
                melspec2 = melspec2.to(device)
                output = model(melspec1, melspec2)
                if output > predVal:
                    pred = i
                    predVal = output
            label = label.to(device)
            if pred == label:
                correct += 1
            count += 1
            if count % 20 == 0:
                print("Current Count is: {}".format(count))
                print('Accuracy on n way: {}'.format(correct/count))

In [78]:
correct = 0
total = 0
#count
with torch.no_grad():
    for melspec1, testSet, label in test_loader:
        #melspec1,testSet, label = data[0].to(device), data[1].to(device), data[2].to(device)
            #melspec1, testSet, label = melspec1.to(device), testSet.to(device), label.to(device)
        melspec1 = melspec1.to(device)
        label =label.to(device)
        predVal = 0
        pred = -1
        for i, melspec2 in enumerate(testSet):
                
            melspec2 = melspec2.to(device)
                
            outputs = load_model(melspec1, melspec2)
            if outputs > predVal:
                pred=i
                predVal = outputs
        if pred==label:
            correct+=1
        
        total += label.size(0)
        #correct += (predicted == label).sum().item()
accr = (100 * correct / total)
print('Accuracy of the model network: %d %%' % (accr))

## 1shot learning

Accuracy of the model network: 99 %


In [ ]:
correct = 0
total = 0
#count
with torch.no_grad():
    for melspec1, testSet, label in test_loader:
        #melspec1,testSet, label = data[0].to(device), data[1].to(device), data[2].to(device)
            #melspec1, testSet, label = melspec1.to(device), testSet.to(device), label.to(device)
        melspec1 = melspec1.to(device)
        label =label.to(device)
        predVal = 0
        pred = -1
        for i, melspec2 in enumerate(testSet):
                
            melspec2 = melspec2.to(device)
                
            outputs = load_model(melspec1, melspec2)
            if outputs > predVal:
                pred=i
                predVal = outputs
        if pred==label:
            correct+=1
        
        total += label.size(0)
        #correct += (predicted == label).sum().item()
accr = (100 * correct / total)
print('Accuracy of the model network: %d %%' % (accr))

In [60]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
siameseBaseLine = Net()
siameseBaseLine = siameseBaseLine.to(device)

def count_parameters(model):
    temp = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f'The model architecture:\n\n', model)
    print(f'\nThe model has {temp:,} trainable parameters')
    
count_parameters(siameseBaseLine)

The model architecture:

 Net(
  (conv1): Conv2d(1, 2, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(2, 5, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(5, 10, kernel_size=(1, 1), stride=(1, 1))
  (conv4): Conv2d(10, 16, kernel_size=(1, 1), stride=(1, 1))
  (bn1): BatchNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm2d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout1): Dropout(p=0.1, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=960, out_features=32, bias=True)
  (fcOut): Linear(in_features=32, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

The model has 31,202 trainable parameters


In [61]:
# saving and loading checkpoint mechanisms
def save_checkpoint(save_path, model, optimizer, val_loss):
    if save_path==None:
        return
    save_path = save_path 
    state_dict = {'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict(),
                  'val_loss': val_loss}

    torch.save(state_dict, save_path)

    print(f'Model saved to ==> {save_path}')

def load_checkpoint(model, optimizer):
    save_path = f'siameseNet-batchnorm50.pt'
    state_dict = torch.load(save_path)
    model.load_state_dict(state_dict['model_state_dict'])
    optimizer.load_state_dict(state_dict['optimizer_state_dict'])
    val_loss = state_dict['val_loss']
    print(f'Model loaded from <== {save_path}')
    
    return val_loss

In [62]:
def train(model, train_loader, val_loader, num_epochs, criterion, save_name):
    best_val_loss = float("Inf") 
    train_losses = []
    val_losses = []
    cur_step = 0
    model.to(device)
    for epoch in range(num_epochs):
        running_loss = 0.0
        model.train()
        print("Starting epoch " + str(epoch+1))
        for audio1, audio2, labels in train_loader:
            
            # Forward
            audio1 = audio1.to(device)
            audio2 = audio2.to(device)
            labels = labels.to(device)
            outputs = model(audio1, audio2)
            outputs.to(device)
            loss = criterion(outputs, labels)
            
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)
        train_losses.append(avg_train_loss)
        
        val_running_loss = 0.0
        with torch.no_grad():
            model.to(device)
            model.eval()
            for audio1, audio2, labels in val_loader:
                audio1 = audio1.to(device)
                audio2 = audio2.to(device)
                labels = labels.to(device)
                outputs = model(audio1, audio2)
                outputs.to(device)
                loss = criterion(outputs, labels)
                val_running_loss += loss.item()
        avg_val_loss = val_running_loss / len(val_loader)
        val_losses.append(avg_val_loss)
        
        print('Epoch [{}/{}],Train Loss: {:.4f}, Valid Loss: {:.8f}'
            .format(epoch+1, num_epochs, avg_train_loss, avg_val_loss))
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            save_checkpoint(save_name, model, optimizer, best_val_loss)
    
    print("Finished Training")  
    return train_losses, val_losses  

# evaluation metrics
def evaluation(model, test_loader):
    model.to(device)
    with torch.no_grad():
        
        model.eval()
        correct = 0
        print('Starting Iteration')
        count = 0
        for mainAudio, testSet, label in test_loader:
            mainAudio = mainAudio.to(device)
            predVal = 0
            pred = -1
            for i, testAudio in enumerate(testSet):
                testAudio = testAudio.to(device)
                output = model(mainAudio, testAudio)
                output.to(device)
                
                
                if output > predVal:
                    pred = i
                    predVal = output
            label = label.to(device)
            #pred.to(device)
            if pred == label:
                
                correct += 1
            count += 1
            if count % 20 == 0:
                print("Current Count is: {}".format(count))
                
                print('Accuracy on n way: {}'.format(correct/count))

In [63]:
# actual training
import torch.optim as optim

optimizer = optim.Adam(siameseBaseLine.parameters(), lr = 0.0006)
num_epochs = 2
#criterion = nn.BCEWithLogitsLoss()
criterion = nn.MSELoss()
save_path = 'siameseNet-batchnorm50.pt'
train_losses, val_losses = train(siameseBaseLine, train_loader, val_loader, num_epochs, criterion, save_path)

Starting epoch 1
Epoch [1/2],Train Loss: 0.4478, Valid Loss: 0.38524995
Model saved to ==> siameseNet-batchnorm50.pt
Starting epoch 2
Epoch [2/2],Train Loss: 0.3809, Valid Loss: 0.36068033
Model saved to ==> siameseNet-batchnorm50.pt
Finished Training


In [64]:
# actual training
import torch.optim as optim

optimizer = optim.Adam(siameseBaseLine.parameters(), lr = 0.0006)
num_epochs = 2
#criterion = nn.BCEWithLogitsLoss()
criterion = nn.MSELoss()
save_path = 'siameseNet-batchnorm50.pt'
train_losses, val_losses = train(siameseBaseLine, train_loader, val_loader, num_epochs, criterion, save_path)

Starting epoch 1
Epoch [1/2],Train Loss: 0.3402, Valid Loss: 0.36162573
Model saved to ==> siameseNet-batchnorm50.pt
Starting epoch 2
Epoch [2/2],Train Loss: 0.3239, Valid Loss: 0.32928113
Model saved to ==> siameseNet-batchnorm50.pt
Finished Training


In [65]:
#torch.cuda.set_device(0)

In [66]:
# Evaluation on previously saved models
import torch.optim as optim
load_model = Net().to(device)
load_optimizer = optim.Adam(load_model.parameters(), lr=0.0006)


In [67]:
num_epochs = 10
eval_every = 1000
total_step = len(train_loader)*num_epochs
best_val_loss = load_checkpoint(load_model, load_optimizer)

print(best_val_loss)

Model loaded from <== siameseNet-batchnorm50.pt
0.3292811316174088


In [68]:
evaluate(load_model, test_loader)

KeyboardInterrupt: 

In [ ]:
plt.xlabel('epoch')
plt.ylabel('loss')
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label="Validation Loss")
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.show()

In [ ]:
"""How to use the evaluation n way:

# Set the parameters
testSize = 5000 # how big you want your test size to be
numWay = 4 # how many ways metric

# Create the dataset for it and put it into dataloader
test_set = NWayOneShotEvalSet(categories, root_dir, testSize, numWay, transformations) 
test_loader = torch.utils.data.DataLoader(test_set, batch_size = 1, num_workers = 2)


# Load the models (the name of the loaded model can be changed in the load_checkpoint() function)
load_model = Net().to(device)
load_optimizer = optim.Adam(load_model.parameters(), lr=0.0006)


num_epochs = 10
eval_every = 1000
total_step = len(train_loader)*num_epochs
best_val_loss = load_checkpoint(load_model, load_optimizer)

print(best_val_loss)

# Evaluate from the test loader 

eval(load_model, test_loader)

"""
 

In [136]:
import gc

gc.collect()

torch.cuda.empty_cache()

In [50]:
!nvidia-smi

NameError: name 'nvidia' is not defined

In [37]:
!nvidia-smi

Wed Jun  1 15:14:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 471.41       Driver Version: 471.41       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:02:00.0 Off |                  N/A |
| N/A   50C    P8     8W /  N/A |   1101MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------